In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

In [27]:
from xgboost import XGBClassifier

# read data

In [28]:
df_train = pd.read_csv('../datasets/train.csv')
df_test = pd.read_csv('../datasets/test.csv')

In [29]:
def convert_passengerID_2_index(df: pd.DataFrame):

    df = df.set_index("PassengerId")
    return df

def split_carbin(df: pd.DataFrame):

    for key, value in enumerate(['deck', 'num', 'side']):
        df[f'Carbin_{value}'] = df['Cabin'].apply(lambda x: str(x).split("/")[key] if len(str(x).split("/")) >= key+1 else None)

    df = df.drop('Cabin', axis=1)
    return df

def drop_columns(df: pd.DataFrame, columns:list):
    
    df = df.drop(columns=columns, axis=1)

    return df

In [30]:
df_train_cleansed = (
    df_train.pipe(convert_passengerID_2_index)
    .pipe(split_carbin)
    .pipe(drop_columns, ['Name', 'Carbin_num'])
)

In [31]:
df_test_cleansed = (
    df_test.pipe(convert_passengerID_2_index)
    .pipe(split_carbin)
    .pipe(drop_columns, ['Name', 'Carbin_num'])
)

# split data into X and y

In [32]:
def split_data_2_x_y(df:pd.DataFrame):

    columns = df.columns 
    X_columns = [i for i in columns if i != 'Transported']
    y_column = 'Transported' if 'Transported' in columns else None 

    if y_column:
        return df[X_columns], df[y_column]
    else:
        return df[X_columns], None

In [33]:
X, y = split_data_2_x_y(df_train_cleansed)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [34]:
X_test, _ = split_data_2_x_y(df_test_cleansed)

In [35]:
X_train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Carbin_deck,Carbin_side
PassengerId,,,,,,,,,,,,
1510_01,Mars,False,TRAPPIST-1e,31.0,False,1226.0,0.0,1.0,0.0,0.0,F,S
7253_01,Europa,False,TRAPPIST-1e,26.0,False,0.0,896.0,0.0,690.0,1.0,D,P
4714_01,Earth,True,TRAPPIST-1e,24.0,False,0.0,0.0,0.0,0.0,0.0,G,P
7727_01,Earth,False,55 Cancri e,33.0,False,0.0,0.0,0.0,436.0,224.0,E,S
3237_01,Mars,False,TRAPPIST-1e,21.0,False,1097.0,0.0,80.0,589.0,0.0,D,P


# data pipeline

In [36]:
# create datapipeline functions

In [37]:
num_pipeline = Pipeline([
    ("Fill N/A value", SimpleImputer(strategy="median")),
    ("standardize", StandardScaler())
])

cate_pipeline = Pipeline([
    ("Fill N/A value", SimpleImputer(strategy="most_frequent")),
    ("One Hot Encoder", OneHotEncoder(handle_unknown="ignore"))
])

In [38]:
num_attribs = X_train.select_dtypes(include="float").columns
cat_attribs = X_train.select_dtypes(include="object").columns

In [39]:
# preprocessing = ColumnTransformer([
#     ("num", num_pipeline, make_column_selector(dtype_include=np.number)),
#     ("cat", cate_pipeline, make_column_selector(dtype_include=object))
# ])

In [40]:
preprocessing = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cate_pipeline, cat_attribs)
])

In [41]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7823 entries, 1510_01 to 7775_01
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    7637 non-null   object 
 1   CryoSleep     7624 non-null   object 
 2   Destination   7658 non-null   object 
 3   Age           7659 non-null   float64
 4   VIP           7642 non-null   object 
 5   RoomService   7675 non-null   float64
 6   FoodCourt     7660 non-null   float64
 7   ShoppingMall  7637 non-null   float64
 8   Spa           7669 non-null   float64
 9   VRDeck        7654 non-null   float64
 10  Carbin_deck   7823 non-null   object 
 11  Carbin_side   7647 non-null   object 
dtypes: float64(6), object(6)
memory usage: 794.5+ KB


# Train the model

In [42]:
# Randome Forest Tree
rnd_clf = make_pipeline(preprocessing, RandomForestClassifier(random_state=42))
rnd_clf.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardize',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('One Hot '
                                                                   'Encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Carbin_deck',
       'Carbin_side'],
      dtype='object'))])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=42))])

In [122]:
# SVC
svc_clf = make_pipeline(preprocessing, SVC(C=5, probability=True, random_state=42))
svc_clf.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardize',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('One Hot '
                                                                   'Encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Carbin_deck',
       'Carbin_side'],
      dtype='object'))])),
                ('svc', SVC(C=5, probability=True, random_state=42))])

In [44]:
# knn
knn = make_pipeline(preprocessing, KNeighborsClassifier(n_neighbors = 3))
knn.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardize',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('One Hot '
                                                                   'Encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Carbin_deck',
       'Carbin_side'],
      dtype='object'))])),
                ('kneighborsclassifier', KNeighborsClassifier(n_neighbors=3))])

In [45]:
# xgboost
xg_clf = make_pipeline(preprocessing, XGBClassifier(learning_rate=0.1, max_depth=3, n_estimators=200, gamma=0.2))
xg_clf.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardize',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('...
                               feature_types=None, gamma=0.2, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=3, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=200,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=None, ...))])

# Tunning the model

### GridSearch CV - Randome Forest Tree

In [46]:
full_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("random_forest", RandomForestClassifier(random_state=42))
])

# param_grid = [
#     {'preprocessing__geo__n_clusters': [5, 8, 10],
#     'random_forest__max_features': [4, 6, 8]},
#     {'preprocessing__geo__n_clusters': [10, 15],
#      'random_forest__max_features': [6, 8, 10]}
# ]
param_grid = [
    {
    'random_forest__max_features': [6, 8, 10],
    'random_forest__n_estimators': [50, 100, 200, 500],
    'random_forest__max_depth': [5, 10, None],
    'random_forest__min_samples_split': [2, 5, 10],
    'random_forest__min_samples_leaf': [1, 2, 4],
    # 'random_forest__max_features': ['auto', 'sqrt']
    }
]
grid_search_rnd_clf = GridSearchCV(full_pipeline, param_grid, cv=3,
                           scoring="f1", #"roc_auc", 
                           n_jobs=-1)
grid_search_rnd_clf.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('Fill '
                                                                                          'N/A '
                                                                                          'value',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardize',
                                                                                          StandardScaler())]),
                                                                         Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('Fill '
                                                                                          'N/A '
                                                                                          'value',
                                                                                          SimpleImputer(s...
                                                                         Index(['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Carbin_deck',
       'Carbin_side'],
      dtype='object'))])),
                                       ('random_forest',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid=[{'random_forest__max_depth': [5, 10, None],
                          'random_forest__max_features': [6, 8, 10],
                          'random_forest__min_samples_leaf': [1, 2, 4],
                          'random_forest__min_samples_split': [2, 5, 10],
                          'random_forest__n_estimators': [50, 100, 200, 500]}],
             scoring='f1')

In [47]:
print('Best hyperparameters:', grid_search_rnd_clf.best_params_)
print('Best score:', grid_search_rnd_clf.best_score_)

Best hyperparameters: {'random_forest__max_depth': 10, 'random_forest__max_features': 8, 'random_forest__min_samples_leaf': 4, 'random_forest__min_samples_split': 10, 'random_forest__n_estimators': 500}
Best score: 0.8096567460317461


In [48]:
cv_res = pd.DataFrame(grid_search_rnd_clf.cv_results_)
cv_res.sort_values(by='mean_test_score', ascending=False, inplace=True)
cv_res.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_random_forest__max_depth,param_random_forest__max_features,param_random_forest__min_samples_leaf,param_random_forest__min_samples_split,param_random_forest__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
179,1.724708,0.014384,0.112398,0.003455,10,8,4,10,500,"{'random_forest__max_depth': 10, 'random_fores...",0.808780,0.810667,0.809524,0.809657,0.000776,1
200,0.198045,0.005042,0.026727,0.003219,10,10,2,10,50,"{'random_forest__max_depth': 10, 'random_fores...",0.809399,0.805303,0.810790,0.808498,0.002329,2
117,0.292146,0.005037,0.025916,0.001825,10,6,1,10,100,"{'random_forest__max_depth': 10, 'random_fores...",0.807952,0.804147,0.812715,0.808272,0.003505,3
115,1.469712,0.005383,0.116261,0.009456,10,6,1,5,500,"{'random_forest__max_depth': 10, 'random_fores...",0.812758,0.802308,0.808920,0.807995,0.004316,4
119,1.510651,0.072630,0.116153,0.009248,10,6,1,10,500,"{'random_forest__max_depth': 10, 'random_fores...",0.809023,0.804465,0.810458,0.807982,0.002555,5


### GridSearch CV - XGBoost

In [104]:
full_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("xg", XGBClassifier(learning_rate=0.1, max_depth=3, n_estimators=200, gamma=0.2))
])

param_grid = [
    {
    'xg__learning_rate': [0.01, 0.1, 0.5],
    'xg__max_depth': [3, 4, 5],
    'xg__n_estimators': [100, 200, 500],
    'xg__gamma': [0.05, 0.2, 1]
    }
]
grid_search_xg = GridSearchCV(full_pipeline, param_grid, cv=3,
                           scoring="f1", #"roc_auc", 
                           n_jobs=-1)
grid_search_xg.fit(X_train, y_train)

/Users/liuyifu/miniconda3/envs/xgboost-env/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('Fill '
                                                                                          'N/A '
                                                                                          'value',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardize',
                                                                                          StandardScaler())]),
                                                                         Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('Fill '
                                                                                          'N/A '
                                                                                          'value',
                                                                                          SimpleImputer(s...
                                                      max_delta_step=None,
                                                      max_depth=3,
                                                      max_leaves=None,
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      n_estimators=200,
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      predictor=None,
                                                      random_state=None, ...))]),
             n_jobs=-1,
             param_grid=[{'xg__gamma': [0.05, 0.2, 1],
                          'xg__learning_rate': [0.01, 0.1, 0.5],
                          'xg__max_depth': [3, 4, 5],
                          'xg__n_estimators': [100, 200, 500]}],
             scoring='f1')

In [105]:
print('Best hyperparameters:', grid_search_xg.best_params_)
print('Best score:', grid_search_xg.best_score_)

Best hyperparameters: {'xg__gamma': 0.05, 'xg__learning_rate': 0.1, 'xg__max_depth': 4, 'xg__n_estimators': 100}
Best score: 0.8162870097323522


In [106]:
cv_res = pd.DataFrame(grid_search_xg.cv_results_)
cv_res.sort_values(by='mean_test_score', ascending=False, inplace=True)
cv_res.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_xg__gamma,param_xg__learning_rate,param_xg__max_depth,param_xg__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
12,0.432879,0.021772,0.008225,0.000728,0.05,0.1,4,100,"{'xg__gamma': 0.05, 'xg__learning_rate': 0.1, ...",0.816932,0.811530,0.820399,0.816287,0.003649,1
39,0.409676,0.019413,0.007447,0.000234,0.2,0.1,4,100,"{'xg__gamma': 0.2, 'xg__learning_rate': 0.1, '...",0.818083,0.813697,0.817037,0.816272,0.001871,2
10,0.599106,0.019389,0.007982,0.000243,0.05,0.1,3,200,"{'xg__gamma': 0.05, 'xg__learning_rate': 0.1, ...",0.816341,0.814212,0.818182,0.816245,0.001622,3
37,0.596459,0.015591,0.011598,0.003157,0.2,0.1,3,200,"{'xg__gamma': 0.2, 'xg__learning_rate': 0.1, '...",0.815751,0.815390,0.816236,0.815793,0.000347,4
66,0.397337,0.002854,0.011796,0.003076,1,0.1,4,100,"{'xg__gamma': 1, 'xg__learning_rate': 0.1, 'xg...",0.813067,0.813397,0.819026,0.815164,0.002735,5


### GridSearch CV - SVC

In [127]:
full_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("svc", SVC(kernel='rbf', probability=True, random_state=42))
])

param_grid = [
    {
    'svc__gamma': [0.005, 0.01, 0.5],
    'svc__C': [10, 15, 20],
    'svc__coef0': [0.1, 0.5, 1],
    'svc__degree': [3, 5]
    }
]
grid_search_svc = GridSearchCV(full_pipeline, param_grid, cv=3,
                           scoring="f1", #"roc_auc", 
                           n_jobs=-1)
grid_search_svc.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('Fill '
                                                                                          'N/A '
                                                                                          'value',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardize',
                                                                                          StandardScaler())]),
                                                                         Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('Fill '
                                                                                          'N/A '
                                                                                          'value',
                                                                                          SimpleImputer(s...t')),
                                                                                         ('One '
                                                                                          'Hot '
                                                                                          'Encoder',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         Index(['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Carbin_deck',
       'Carbin_side'],
      dtype='object'))])),
                                       ('svc',
                                        SVC(probability=True,
                                            random_state=42))]),
             n_jobs=-1,
             param_grid=[{'svc__C': [10, 15, 20], 'svc__coef0': [0.1, 0.5, 1],
                          'svc__degree': [3, 5],
                          'svc__gamma': [0.005, 0.01, 0.5]}],
             scoring='f1')

In [130]:
print('Best hyperparameters:', grid_search_svc.best_params_)
print('Best score:', grid_search_svc.best_score_)

Best hyperparameters: {'svc__C': 20, 'svc__coef0': 0.1, 'svc__degree': 3, 'svc__gamma': 0.01}
Best score: 0.807791513000037


In [131]:
cv_res = pd.DataFrame(grid_search_svc.cv_results_)
cv_res.sort_values(by='mean_test_score', ascending=False, inplace=True)
cv_res.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svc__C,param_svc__coef0,param_svc__degree,param_svc__gamma,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
43,2.427713,0.038498,0.338494,0.045335,20,0.5,3,0.01,"{'svc__C': 20, 'svc__coef0': 0.5, 'svc__degree...",0.807185,0.800605,0.815584,0.807792,0.00613,1
52,2.366757,0.026857,0.284709,0.009868,20,1,5,0.01,"{'svc__C': 20, 'svc__coef0': 1, 'svc__degree':...",0.807185,0.800605,0.815584,0.807792,0.00613,1
37,2.537150,0.008273,0.327131,0.006314,20,0.1,3,0.01,"{'svc__C': 20, 'svc__coef0': 0.1, 'svc__degree...",0.807185,0.800605,0.815584,0.807792,0.00613,1
49,2.328250,0.035163,0.328520,0.008703,20,1,3,0.01,"{'svc__C': 20, 'svc__coef0': 1, 'svc__degree':...",0.807185,0.800605,0.815584,0.807792,0.00613,1
40,2.366577,0.022389,0.306628,0.012220,20,0.1,5,0.01,"{'svc__C': 20, 'svc__coef0': 0.1, 'svc__degree...",0.807185,0.800605,0.815584,0.807792,0.00613,1


## Stacking

In [136]:
estimators = [
    ('rnd_best_clf', RandomForestClassifier(max_depth=10, max_features=10, min_samples_leaf=4,
                                       min_samples_split=10, n_estimators =100,
                                       random_state=42)),
    ('svc', SVC(probability=True, random_state=42)),
    ('best_svc', SVC(C=20, coef0=0.1, degree=3, gamma=0.01, probability=True)),
    ('rnd_clf', RandomForestClassifier(random_state=42)),
    ('xgboost', XGBClassifier(gamma=0.05, learning_rate=0.1, max_depth=4, n_estimators=100))
    
]

stacking_classifer = StackingClassifier(
    estimators=estimators,
    final_estimator=XGBClassifier(learning_rate=0.1, max_depth=3, n_estimators=200, gamma=0.2),
    cv=5,
    n_jobs=-1
)
stacking_clf = Pipeline([
    ('preprocessing', preprocessing),
    ('stacking_classifier', stacking_classifer)
])
stacking_clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardize',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('One H...
                                                                  grow_policy=None,
                                                                  importance_type=None,
                                                                  interaction_constraints=None,
                                                                  learning_rate=0.1,
                                                                  max_bin=None,
                                                                  max_cat_threshold=None,
                                                                  max_cat_to_onehot=None,
                                                                  max_delta_step=None,
                                                                  max_depth=3,
                                                                  max_leaves=None,
                                                                  min_child_weight=None,
                                                                  missing=nan,
                                                                  monotone_constraints=None,
                                                                  n_estimators=200,
                                                                  n_jobs=None,
                                                                  num_parallel_tree=None,
                                                                  predictor=None,
                                                                  random_state=None, ...),
                                    n_jobs=-1))])

# Voting

In [137]:
estimators = [
    ('rnd_best_clf', RandomForestClassifier(max_depth=10, max_features=10, min_samples_leaf=4,
                                       min_samples_split=10, n_estimators =100,
                                       random_state=42)),
    ('svc', SVC(C=5, probability=True, random_state=42)),
    ('best_svc', SVC(C=20, coef0=0.1, degree=3, gamma=0.01, probability=True)),
    ('rnd_clf', RandomForestClassifier(random_state=42)),
    ('xgboost', XGBClassifier(gamma=0.05, learning_rate=0.1, max_depth=4, n_estimators=100))
]

voting_classifer = VotingClassifier(
    estimators=estimators,
    voting='soft',
    n_jobs=-1
)
voting_clf = Pipeline([
    ('preprocessing', preprocessing),
    ('stacking_classifier', stacking_classifer)
])
voting_clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardize',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('One H...
                                                                  grow_policy=None,
                                                                  importance_type=None,
                                                                  interaction_constraints=None,
                                                                  learning_rate=0.1,
                                                                  max_bin=None,
                                                                  max_cat_threshold=None,
                                                                  max_cat_to_onehot=None,
                                                                  max_delta_step=None,
                                                                  max_depth=3,
                                                                  max_leaves=None,
                                                                  min_child_weight=None,
                                                                  missing=nan,
                                                                  monotone_constraints=None,
                                                                  n_estimators=200,
                                                                  n_jobs=None,
                                                                  num_parallel_tree=None,
                                                                  predictor=None,
                                                                  random_state=None, ...),
                                    n_jobs=-1))])

# Evaluate the performance

## Evaluate the performance on the training set

In [54]:
# Randome Forest Tree
y_proba = rnd_clf.predict_proba(X_train)[:, 1]
y_pred = rnd_clf.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.9884393152376948
F1 score: 0.9496027805362464


In [123]:
# SVC
y_proba = svc_clf.predict_proba(X_train)[:, 1]
y_pred = svc_clf.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.9014993981019556
F1 score: 0.825731875863802


In [56]:
# SVC Grid Search CV
y_proba = grid_search_svc.predict_proba(X_train)[:, 1]
y_pred = grid_search_svc.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.891205915282458
F1 score: 0.812344449975112


In [57]:
# Randome Forest Tree Grid Search CV
y_proba = grid_search_rnd_clf.predict_proba(X_train)[:, 1]
y_pred = grid_search_rnd_clf.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.9303950145480484
F1 score: 0.8432609793033821


In [58]:
# knn
y_proba = knn.predict_proba(X_train)[:, 1]
y_pred = knn.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.9302028032969998
F1 score: 0.8524173027989822


In [99]:
# xgboost
y_proba = xg_clf.predict_proba(X_train)[:, 1]
y_pred = xg_clf.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.9130067108232877
F1 score: 0.833088954056696


In [107]:
# Search CV - xgboost

y_proba = grid_search_xg.predict_proba(X_train)[:, 1]
y_pred = grid_search_xg.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.9132601381384774
F1 score: 0.8310636731689363


In [138]:
# Stacking
y_proba = stacking_clf.predict_proba(X_train)[:, 1]
y_pred = stacking_clf.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)


AUC-ROC score: 0.920333420663458
F1 score: 0.8361696723393447


In [139]:
# voting
y_proba = voting_clf.predict_proba(X_train)[:, 1]
y_pred = voting_clf.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)


AUC-ROC score: 0.920333420663458
F1 score: 0.8361696723393447


### Analyse Features

In [62]:
grid_search_model = grid_search_rnd_clf.best_estimator_
# feature_importances = grid_search_model
feature_importances = grid_search_model["random_forest"].feature_importances_
feature_importances.round(4)

sorted(zip(feature_importances, grid_search_model["preprocessing"].get_feature_names_out()), reverse=True)[:20]

[(0.1503001874823897, 'cat__CryoSleep_False'),
 (0.13039033127990787, 'cat__CryoSleep_True'),
 (0.1280785859581798, 'num__Spa'),
 (0.10835650626332144, 'num__VRDeck'),
 (0.10242614220101531, 'num__RoomService'),
 (0.08417259439593829, 'num__FoodCourt'),
 (0.0690316605703463, 'num__Age'),
 (0.05861438075127716, 'num__ShoppingMall'),
 (0.03478914361702934, 'cat__HomePlanet_Earth'),
 (0.02414412677935901, 'cat__HomePlanet_Europa'),
 (0.01922984010041955, 'cat__Carbin_deck_G'),
 (0.014952354611416818, 'cat__Carbin_deck_E'),
 (0.01256685291239199, 'cat__Carbin_deck_F'),
 (0.011822883067355327, 'cat__Carbin_side_S'),
 (0.011623633350910694, 'cat__Carbin_side_P'),
 (0.008596070311784753, 'cat__HomePlanet_Mars'),
 (0.006690086229399202, 'cat__Carbin_deck_C'),
 (0.006287712463655947, 'cat__Destination_55 Cancri e'),
 (0.006132383188942292, 'cat__Destination_TRAPPIST-1e'),
 (0.004038581980955024, 'cat__Carbin_deck_B')]

## Evaluate the performance on the validation set

In [63]:
# randome forest tree
y_proba = rnd_clf.predict_proba(X_val)[:, 1]
y_pred = rnd_clf.predict(X_val)
auc_roc = roc_auc_score(y_val, y_proba)
f1 = f1_score(y_val, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.8614962915188649
F1 score: 0.768867924528302


In [133]:
# SVC
y_proba = svc_clf.predict_proba(X_val)[:, 1]
y_pred = svc_clf.predict(X_val)
auc_roc = roc_auc_score(y_val, y_proba)
f1 = f1_score(y_val, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.86673785822659
F1 score: 0.7913832199546486


In [134]:
# gridsearchCV - SVC
y_proba = grid_search_svc.predict_proba(X_val)[:, 1]
y_pred = grid_search_svc.predict(X_val)
auc_roc = roc_auc_score(y_val, y_proba)
f1 = f1_score(y_val, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.8678427371392623
F1 score: 0.7853170189098999


In [66]:
# gridsearchCV - Random Forest Tree
y_proba = grid_search_rnd_clf.predict_proba(X_val)[:, 1]
y_pred = grid_search_rnd_clf.predict(X_val)
auc_roc = roc_auc_score(y_val, y_proba)
f1 = f1_score(y_val, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.8784316005942028
F1 score: 0.7867730900798175


In [67]:
# knn
y_proba = knn.predict_proba(X_val)[:, 1]
y_pred = knn.predict(X_val)
auc_roc = roc_auc_score(y_val, y_proba)
f1 = f1_score(y_val, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.8459724784707207
F1 score: 0.7617924528301887


In [115]:
# xgboost
y_proba = xg_clf.predict_proba(X_train)[:, 1]
y_pred = xg_clf.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.9130067108232877
F1 score: 0.833088954056696


In [108]:
# searchgrid cv - xgboost
y_proba = grid_search_xg.predict_proba(X_train)[:, 1]
y_pred = grid_search_xg.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.9132601381384774
F1 score: 0.8310636731689363


In [140]:
# Stacking
y_proba = stacking_clf.predict_proba(X_val)[:, 1]
y_pred = stacking_clf.predict(X_val)
auc_roc = roc_auc_score(y_val, y_proba)
f1 = f1_score(y_val, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)


AUC-ROC score: 0.8775646142703833
F1 score: 0.7894736842105264


In [141]:
# voting
y_proba = voting_clf.predict_proba(X_val)[:, 1]
y_pred = voting_clf.predict(X_val)
auc_roc = roc_auc_score(y_val, y_proba)
f1 = f1_score(y_val, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)


AUC-ROC score: 0.8775646142703833
F1 score: 0.7894736842105264


# output data

In [71]:
# randome forest tree
y_test = rnd_clf.predict(X_test)
output = pd.DataFrame(y_test, index=X_test.index, columns=['Transported'])
output.to_csv('submission.csv') # 0.7912

In [126]:
# SVC
y_test = svc_clf.predict(X_test)
output = pd.DataFrame(y_test, index=X_test.index, columns=['Transported'])
output.to_csv('submission.csv') # 0.80406

In [87]:
# gridsearchCV - SVC
y_test = grid_search_svc.predict(X_test)
output = pd.DataFrame(y_test, index=X_test.index, columns=['Transported'])
output.to_csv('submission.csv') # 0.80173

In [74]:
# gridsearchCV - Random Forest Tree
y_test = grid_search_rnd_clf.predict(X_test)
output = pd.DataFrame(y_test, index=X_test.index, columns=['Transported'])
output.to_csv('submission.csv') # 0.7877

In [80]:
# knn
y_test = knn.predict(X_test)
output = pd.DataFrame(y_test, index=X_test.index, columns=['Transported'])
output.to_csv('submission.csv')

In [88]:
# xgboost
y_test = xg_clf.predict(X_test)
y_test = [[False, True][i] for i in y_test ]
output = pd.DataFrame(y_test, index=X_test.index, columns=['Transported'])
output.to_csv('submission.csv') # 0.80243

In [110]:
# Search CV - xgboost
y_test = grid_search_xg.predict(X_test)
y_test = [[False, True][i] for i in y_test ]
output = pd.DataFrame(y_test, index=X_test.index, columns=['Transported'])
output.to_csv('submission.csv') 

In [142]:
# stacking
y_test = stacking_clf.predict(X_test)
output = pd.DataFrame(y_test, index=X_test.index, columns=['Transported'])
output.to_csv('submission.csv') # 0.79939

In [121]:
# voting
y_test = voting_clf.predict(X_test)
output = pd.DataFrame(y_test, index=X_test.index, columns=['Transported'])
output.to_csv('submission.csv') 